In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model, preprocessing
from sklearn.model_selection import train_test_split
# from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler


In [76]:
# Loading the dataset

file_path = '../cl_JUIN_2013-complet3.csv'  # Use '..' to go up one level

df = pd.read_csv(file_path, encoding='ISO-8859-1', sep=';')

df.head(10)

,Marque,Modèle dossier,Modèle UTAC,Désignation commerciale,CNIT,Type Variante Version (TVV),Carburant,Hybride,Puissance administrative,Puissance maximale (kW),...,HC (g/km),NOX (g/km),HC+NOX (g/km),Particules (g/km),masse vide euro min (kg),masse vide euro max (kg),Champ V9,Date de mise à jour,Carrosserie,gamme
0,ALFA-ROMEO,159,159,159 1750 Tbi (200ch),M10ALFVP000G340,939AXN1B52C,ES,non,12,147.0,...,0.052,0.032,NaN,0.002,1505,1505,715/2007*692/2008EURO5,juin-13,BREAK,MOY-SUPER
1,ALFA-ROMEO,159,159,159 2.0 JTDm (170ch) ECO,M10ALFVP000U221,939AXP1B54C,GO,non,9,125.0,...,NaN,0.169,0.190,0.003,1565,1565,715/2007*692/2008EURO5,juin-13,BERLINE,MOY-SUPER
2,ALFA-ROMEO,159,159,159 2.0 JTDm (136ch),M10ALFVP000E302,939AXR1B64,GO,non,7,100.0,...,NaN,0.149,0.175,0.001,1565,1565,715/2007*692/2008EURO5,juin-13,BERLINE,MOY-SUPER
3,ALFA-ROMEO,159,159,159 2.0 JTDm (136ch),M10ALFVP000F303,939AXR1B64B,GO,non,7,100.0,...,NaN,0.149,0.175,0.001,1565,1565,715/2007*692/2008EURO5,juin-13,BERLINE,MOY-SUPER
4,ALFA-ROMEO,159,159,159 2.0 JTDm (170ch),M10ALFVP000G304,939AXS1B66,GO,non,9,125.0,...,NaN,0.164,0.193,0.001,1565,1565,715/2007*692/2008EURO5,juin-13,BERLINE,MOY-SUPER
5,ALFA-ROMEO,159,159,159 2.0 JTDm (170ch),M10ALFVP000H305,939AXS1B66B,GO,non,9,125.0,...,NaN,0.164,0.193,0.001,1565,1565,715/2007*692/2008EURO5,juin-13,BERLINE,MOY-SUPER
6,ALFA-ROMEO,159,159,159 1750 Tbi (200ch),M10ALFVP000H341,939BXN1B53C,ES,non,12,147.0,...,0.052,0.032,NaN,0.002,1555,1555,715/2007*692/2008EURO5,juin-13,BREAK,MOY-SUPER
7,ALFA-ROMEO,159,159,159 SW 2.0 JTDm (170ch) ECO,M10ALFVP000S255,939BXP1B55C,GO,non,9,125.0,...,NaN,0.169,0.190,0.003,1615,1615,715/2007*692/2008EURO5,juin-13,BREAK,MOY-SUPER
8,ALFA-ROMEO,159,159,159 SW 2.0 JTDm (136ch),M10ALFVP000J306,939BXR1B65,GO,non,7,100.0,...,NaN,0.149,0.175,0.001,1615,1615,715/2007*692/2008EURO5,juin-13,BREAK,MOY-SUPER
9,ALFA-ROMEO,159,159,159 SW 2.0 JTDm (136ch),M10ALFVP000J307,939BXR1B65B,GO,non,7,100.0,...,NaN,0.149,0.175,0.001,1615,1615,715/2007*692/2008EURO5,juin-13,BREAK,MOY-SUPER


In [77]:
# Drop columns with high percentage of missing values
df.drop(columns=['HC (g/km)'], inplace=True)

In [78]:
from sklearn.impute import KNNImputer

# Selecting only the columns with missing values
cols_with_missing = ['HC+NOX (g/km)']

# Creating a copy of the DataFrame with only the selected columns
data_with_missing = df[cols_with_missing].copy()

# Initialize the KNN imputer
imputer = KNNImputer(n_neighbors=5)

# Fit the imputer to the data and transform it
imputed_data = imputer.fit_transform(data_with_missing)

# Convert the imputed data back to a DataFrame
imputed_df = pd.DataFrame(imputed_data, columns=cols_with_missing)

# Update the original DataFrame with the imputed values
df[cols_with_missing] = imputed_df

df[cols_with_missing].head()

,HC+NOX (g/km)
0,0.224788
1,0.190000
2,0.175000
3,0.175000
4,0.193000


In [79]:
# Impute missing values in the Particules column with moderate percentage of missing values
df['Particules (g/km)'].fillna(df['Particules (g/km)'].median(), inplace=True)

In [80]:
# Impute missing values in columns with low percentage of missing values
numeric_columns_to_impute = ['NOX (g/km)', 'CO type I (g/km)', 
                             'Consommation urbaine (l/100km)', 
                             'Consommation extra-urbaine (l/100km)',
                             'Consommation mixte (l/100km)', 
                             'CO2 (g/km)']

# Impute missing values with the mean of each numeric column
for column in numeric_columns_to_impute:
    df[column].fillna(df[column].mean(), inplace=True)

# Check if there are any missing values left
print(df.isnull().sum())

Marque                                    0
Modèle dossier                            0
Modèle UTAC                               0
Désignation commerciale                   0
CNIT                                      0
Type Variante Version (TVV)               0
Carburant                                 0
Hybride                                   0
Puissance administrative                  0
Puissance maximale (kW)                   0
Boîte de vitesse                          0
Consommation urbaine (l/100km)            0
Consommation extra-urbaine (l/100km)      0
Consommation mixte (l/100km)              0
CO2 (g/km)                                0
CO type I (g/km)                          0
NOX (g/km)                                0
HC+NOX (g/km)                             0
Particules (g/km)                         0
masse vide euro min (kg)                  0
masse vide euro max (kg)                  0
Champ V9                                235
Date de mise à jour             

In [81]:
# Impute missing values in 'Champ V9' with the mode
mode_champ_v9 = df['Champ V9'].mode()[0]
df['Champ V9'].fillna(mode_champ_v9, inplace=True)

# Check if there are any missing values left
print(df.isnull().sum())

Marque                                  0
Modèle dossier                          0
Modèle UTAC                             0
Désignation commerciale                 0
CNIT                                    0
Type Variante Version (TVV)             0
Carburant                               0
Hybride                                 0
Puissance administrative                0
Puissance maximale (kW)                 0
Boîte de vitesse                        0
Consommation urbaine (l/100km)          0
Consommation extra-urbaine (l/100km)    0
Consommation mixte (l/100km)            0
CO2 (g/km)                              0
CO type I (g/km)                        0
NOX (g/km)                              0
HC+NOX (g/km)                           0
Particules (g/km)                       0
masse vide euro min (kg)                0
masse vide euro max (kg)                0
Champ V9                                0
Date de mise à jour                     0
Carrosserie                       

In [82]:
# Removing duplicates
df.drop_duplicates(inplace=True)
df.head(20)

,Marque,Modèle dossier,Modèle UTAC,Désignation commerciale,CNIT,Type Variante Version (TVV),Carburant,Hybride,Puissance administrative,Puissance maximale (kW),...,CO type I (g/km),NOX (g/km),HC+NOX (g/km),Particules (g/km),masse vide euro min (kg),masse vide euro max (kg),Champ V9,Date de mise à jour,Carrosserie,gamme
0,ALFA-ROMEO,159,159,159 1750 Tbi (200ch),M10ALFVP000G340,939AXN1B52C,ES,non,12,147.0,...,0.647,0.032,0.224788,0.002,1505,1505,715/2007*692/2008EURO5,juin-13,BREAK,MOY-SUPER
1,ALFA-ROMEO,159,159,159 2.0 JTDm (170ch) ECO,M10ALFVP000U221,939AXP1B54C,GO,non,9,125.0,...,0.192,0.169,0.190000,0.003,1565,1565,715/2007*692/2008EURO5,juin-13,BERLINE,MOY-SUPER
2,ALFA-ROMEO,159,159,159 2.0 JTDm (136ch),M10ALFVP000E302,939AXR1B64,GO,non,7,100.0,...,0.066,0.149,0.175000,0.001,1565,1565,715/2007*692/2008EURO5,juin-13,BERLINE,MOY-SUPER
3,ALFA-ROMEO,159,159,159 2.0 JTDm (136ch),M10ALFVP000F303,939AXR1B64B,GO,non,7,100.0,...,0.066,0.149,0.175000,0.001,1565,1565,715/2007*692/2008EURO5,juin-13,BERLINE,MOY-SUPER
4,ALFA-ROMEO,159,159,159 2.0 JTDm (170ch),M10ALFVP000G304,939AXS1B66,GO,non,9,125.0,...,0.060,0.164,0.193000,0.001,1565,1565,715/2007*692/2008EURO5,juin-13,BERLINE,MOY-SUPER
5,ALFA-ROMEO,159,159,159 2.0 JTDm (170ch),M10ALFVP000H305,939AXS1B66B,GO,non,9,125.0,...,0.060,0.164,0.193000,0.001,1565,1565,715/2007*692/2008EURO5,juin-13,BERLINE,MOY-SUPER
6,ALFA-ROMEO,159,159,159 1750 Tbi (200ch),M10ALFVP000H341,939BXN1B53C,ES,non,12,147.0,...,0.647,0.032,0.224788,0.002,1555,1555,715/2007*692/2008EURO5,juin-13,BREAK,MOY-SUPER
7,ALFA-ROMEO,159,159,159 SW 2.0 JTDm (170ch) ECO,M10ALFVP000S255,939BXP1B55C,GO,non,9,125.0,...,0.192,0.169,0.190000,0.003,1615,1615,715/2007*692/2008EURO5,juin-13,BREAK,MOY-SUPER
8,ALFA-ROMEO,159,159,159 SW 2.0 JTDm (136ch),M10ALFVP000J306,939BXR1B65,GO,non,7,100.0,...,0.066,0.149,0.175000,0.001,1615,1615,715/2007*692/2008EURO5,juin-13,BREAK,MOY-SUPER
9,ALFA-ROMEO,159,159,159 SW 2.0 JTDm (136ch),M10ALFVP000J307,939BXR1B65B,GO,non,7,100.0,...,0.066,0.149,0.175000,0.001,1615,1615,715/2007*692/2008EURO5,juin-13,BREAK,MOY-SUPER


In [83]:

# Filter out the categorical variables
categorical_variables = df.select_dtypes(include=['object', 'category'])

# Create a dictionary to store the number of categories for each variable
categories_count = {}

# Iterate through each categorical variable and count the number of unique categories
for column in categorical_variables.columns:
    categories_count[column] = len(df[column].unique())

# Print the dictionary containing the number of categories for each variable
print(categories_count)

{'Marque': 51, 'Modèle dossier': 458, 'Modèle UTAC': 419, 'Désignation commerciale': 3582, 'CNIT': 44191, 'Type Variante Version (TVV)': 28781, 'Carburant': 13, 'Hybride': 2, 'Boîte de vitesse': 16, 'Champ V9': 13, 'Date de mise à jour': 3, 'Carrosserie': 10, 'gamme': 7}


In [84]:
# Dropping redundant columns
df.drop(['Modèle dossier'], axis=1, inplace=True)

In [85]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# Apply label encoding to the 'Date de mise à jour' and 'Hybride' columns
df['Date de mise à jour'] = label_encoder.fit_transform(df['Date de mise à jour'])
df['Hybride'] = label_encoder.fit_transform(df['Hybride'])
df['CNIT'] = label_encoder.fit_transform(df['CNIT'])
df.head()

,Marque,Modèle UTAC,Désignation commerciale,CNIT,Type Variante Version (TVV),Carburant,Hybride,Puissance administrative,Puissance maximale (kW),Boîte de vitesse,...,CO type I (g/km),NOX (g/km),HC+NOX (g/km),Particules (g/km),masse vide euro min (kg),masse vide euro max (kg),Champ V9,Date de mise à jour,Carrosserie,gamme
0,ALFA-ROMEO,159,159 1750 Tbi (200ch),56,939AXN1B52C,ES,0,12,147.0,M 6,...,0.647,0.032,0.224788,0.002,1505,1505,715/2007*692/2008EURO5,1,BREAK,MOY-SUPER
1,ALFA-ROMEO,159,159 2.0 JTDm (170ch) ECO,92,939AXP1B54C,GO,0,9,125.0,M 6,...,0.192,0.169,0.190000,0.003,1565,1565,715/2007*692/2008EURO5,1,BERLINE,MOY-SUPER
2,ALFA-ROMEO,159,159 2.0 JTDm (136ch),45,939AXR1B64,GO,0,7,100.0,M 6,...,0.066,0.149,0.175000,0.001,1565,1565,715/2007*692/2008EURO5,1,BERLINE,MOY-SUPER
3,ALFA-ROMEO,159,159 2.0 JTDm (136ch),50,939AXR1B64B,GO,0,7,100.0,M 6,...,0.066,0.149,0.175000,0.001,1565,1565,715/2007*692/2008EURO5,1,BERLINE,MOY-SUPER
4,ALFA-ROMEO,159,159 2.0 JTDm (170ch),55,939AXS1B66,GO,0,9,125.0,M 6,...,0.060,0.164,0.193000,0.001,1565,1565,715/2007*692/2008EURO5,1,BERLINE,MOY-SUPER


In [86]:
# Apply one-hot encoding for the variable 'gamme'
df = pd.get_dummies(df, columns=['gamme'])


In [87]:
# Apply one-hot encoding for the variable 'Marque'
df = pd.get_dummies(df, columns=['Marque'])
# Apply one-hot encoding for the variable 'Carburant'
df = pd.get_dummies(df, columns=['Carburant'])
# Apply one-hot encoding for the variable 'Boîte de vitesse'
df = pd.get_dummies(df, columns=['Boîte de vitesse'])
# Apply one-hot encoding for the variable 'Carrosserie'
df = pd.get_dummies(df, columns=['Carrosserie'])



In [88]:
# Replace "True" with 1 and "False" with 0 in all columns
df = df.replace({True: 1, False: 0})
df.head()

,Modèle UTAC,Désignation commerciale,CNIT,Type Variante Version (TVV),Hybride,Puissance administrative,Puissance maximale (kW),Consommation urbaine (l/100km),Consommation extra-urbaine (l/100km),Consommation mixte (l/100km),...,Carrosserie_BERLINE,Carrosserie_BREAK,Carrosserie_CABRIOLET,Carrosserie_COMBISPACE,Carrosserie_COUPE,Carrosserie_MINIBUS,Carrosserie_MINISPACE,Carrosserie_MONOSPACE,Carrosserie_MONOSPACE COMPACT,Carrosserie_TS TERRAINS/CHEMINS
0,159,159 1750 Tbi (200ch),56,939AXN1B52C,0,12,147.0,11.3,5.8,7.8,...,0,1,0,0,0,0,0,0,0,0
1,159,159 2.0 JTDm (170ch) ECO,92,939AXP1B54C,0,9,125.0,6.6,4.3,5.1,...,1,0,0,0,0,0,0,0,0,0
2,159,159 2.0 JTDm (136ch),45,939AXR1B64,0,7,100.0,6.6,4.2,5.1,...,1,0,0,0,0,0,0,0,0,0
3,159,159 2.0 JTDm (136ch),50,939AXR1B64B,0,7,100.0,6.6,4.2,5.1,...,1,0,0,0,0,0,0,0,0,0
4,159,159 2.0 JTDm (170ch),55,939AXS1B66,0,9,125.0,6.9,4.3,5.3,...,1,0,0,0,0,0,0,0,0,0


In [89]:
# Transforming 'Puissance administrative' into a categorical feature

# what do you thing about?
# Creating a feature based on power to weight ratio
#df['power_to_weight'] = df['Puissance maximale (kW)'] / df['masse vide euro max (kg)']
#df['Puissance_cat'] = pd.cut(df['Puissance administrative'], bins=[0, 7, 14, 20], labels=['Low', 'Medium', 'High'])

#df.head()

In [90]:
# Encoding categorical variables

#categorical_features = ['Marque', 'Carburant', 'Hybride', 'Puissance_cat']
#df_encoded = pd.get_dummies(df, columns=categorical_features)

In [91]:
# Normalizing numerical features

numerical_features = df.select_dtypes(include=['int64', 'float64']).columns
scaler = MinMaxScaler()
df[numerical_features] = scaler.fit_transform(df[numerical_features])

In [92]:
df.head(20)

,Modèle UTAC,Désignation commerciale,CNIT,Type Variante Version (TVV),Hybride,Puissance administrative,Puissance maximale (kW),Consommation urbaine (l/100km),Consommation extra-urbaine (l/100km),Consommation mixte (l/100km),...,Carrosserie_BERLINE,Carrosserie_BREAK,Carrosserie_CABRIOLET,Carrosserie_COMBISPACE,Carrosserie_COUPE,Carrosserie_MINIBUS,Carrosserie_MINISPACE,Carrosserie_MONOSPACE,Carrosserie_MONOSPACE COMPACT,Carrosserie_TS TERRAINS/CHEMINS
0,159,159 1750 Tbi (200ch),56,939AXN1B52C,0,0.1375,0.249408,0.274939,0.247934,0.283262,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,159,159 2.0 JTDm (170ch) ECO,92,939AXP1B54C,0,0.1000,0.209357,0.160584,0.123967,0.167382,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,159,159 2.0 JTDm (136ch),45,939AXR1B64,0,0.0750,0.163845,0.160584,0.115702,0.167382,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,159,159 2.0 JTDm (136ch),50,939AXR1B64B,0,0.0750,0.163845,0.160584,0.115702,0.167382,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,159,159 2.0 JTDm (170ch),55,939AXS1B66,0,0.1000,0.209357,0.167883,0.123967,0.175966,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,159,159 2.0 JTDm (170ch),58,939AXS1B66B,0,0.1000,0.209357,0.167883,0.123967,0.175966,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,159,159 1750 Tbi (200ch),59,939BXN1B53C,0,0.1375,0.249408,0.279805,0.264463,0.291845,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,159,159 SW 2.0 JTDm (170ch) ECO,87,939BXP1B55C,0,0.1000,0.209357,0.163017,0.132231,0.171674,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,159,159 SW 2.0 JTDm (136ch),62,939BXR1B65,0,0.0750,0.163845,0.165450,0.123967,0.171674,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,159,159 SW 2.0 JTDm (136ch),63,939BXR1B65B,0,0.0750,0.163845,0.165450,0.123967,0.171674,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [104]:
from scipy.stats import zscore

# Select numerical columns
numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns

# Calculate z-scores for each numerical column
z_scores = df[numerical_columns].apply(lambda x: zscore(x))

# Define a threshold for identifying outliers
threshold = 3

# Create a mask of outliers
outlier_mask = (z_scores.abs() > threshold)

# Winsorize outliers by capping them at a certain percentile
lower_quantile = df[numerical_columns].quantile(0.01, axis=0)
upper_quantile = df[numerical_columns].quantile(0.99, axis=0)
winsorized_values = df[numerical_columns].clip(lower=lower_quantile, upper=upper_quantile, axis=1)

# Replace outliers with winsorized values
df[numerical_columns][outlier_mask] = winsorized_values[outlier_mask]

# Print the number of outliers in each column after treatment
# Print the full list of column names and their respective number of outliers
print("Number of outliers in each column after treatment:")
for column, count in outlier_counts_df.iterrows():
    print(f"{column}: {count['Number of Outliers']}")







Number of outliers in each column after treatment:
Puissance administrative: 1041
Puissance maximale (kW): 1049
Consommation urbaine (l/100km): 504
Consommation extra-urbaine (l/100km): 302
Consommation mixte (l/100km): 394
CO2 (g/km): 263
CO type I (g/km): 573
NOX (g/km): 3644
HC+NOX (g/km): 617
Particules (g/km): 59
masse vide euro min (kg): 298
masse vide euro max (kg): 122
gamme_ECONOMIQUE: 219
gamme_INFERIEURE: 1622
gamme_LUXE: 0
gamme_MOY-INF: 2
gamme_MOY-INFER: 0
gamme_MOY-SUPER: 0
gamme_SUPERIEURE: 1956
Marque_ALFA-ROMEO: 103
Marque_ASTON MARTIN: 78
Marque_AUDI: 242
Marque_BENTLEY: 22
Marque_BMW: 525
Marque_CADILLAC: 44
Marque_CHEVROLET: 63
Marque_CITROEN: 207
Marque_DACIA: 30
Marque_DANGEL: 40
Marque_FERRARI: 21
Marque_FIAT: 415
Marque_FORD: 296
Marque_HONDA: 51
Marque_HYUNDAI: 52
Marque_INFINITI: 26
Marque_JAGUAR: 43
Marque_JAGUAR LAND ROVER LIMITED: 55
Marque_JEEP: 74
Marque_KIA: 78
Marque_LADA: 9
Marque_LAMBORGHINI: 16
Marque_LANCIA: 37
Marque_LEXUS: 175
Marque_LOTUS: 26
Ma

C:\Users\Halimeh\AppData\Local\Temp\ipykernel_11552\1840596793.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[numerical_columns][outlier_mask] = winsorized_values[outlier_mask]
C:\Users\Halimeh\AppData\Local\Temp\ipykernel_11552\1840596793.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[numerical_columns][outlier_mask] = winsorized_values[outlier_mask]


In [94]:
from sklearn.model_selection import train_test_split

# Splitting the data into features (X) and target variable (y)
X = df.drop(columns=['CO2 (g/km)'])
y = df['CO2 (g/km)']

# Splitting the data into training and remaining data
X_train, X_remaining, y_train, y_remaining = train_test_split(X, y, test_size=0.2, random_state=42)

# Further splitting the remaining data into validation and test sets
X_val, X_test, y_val, y_test = train_test_split(X_remaining, y_remaining, test_size=0.5, random_state=42)



In [74]:
# Save the DataFrame to a new CSV file
df.to_csv('Clean_Data_Preprocessed.csv', index=False)